In [ ]:
!pip install vnstock

In [2]:
import pandas as pd
import os
import requests
import json
import time
import shutil
from vnstock import *


In [ ]:
tickers = listing_companies()["ticker"]

In [ ]:
df =  stock_historical_data(symbol='GMD', 
                            start_date="2011-01-01", 
                            end_date='2012-01-01')
df.to_csv("/content/test.csv")

        open     high      low    close  volume               tradingDate
0    12213.0  12213.0  11924.0  11924.0  152600  2011-01-04T00:00:00.000Z
1    11924.0  11924.0  11635.0  11635.0  322840  2011-01-05T00:00:00.000Z
2    11490.0  11635.0  11454.0  11454.0  350890  2011-01-06T00:00:00.000Z
3    11526.0  11671.0  11454.0  11454.0  182450  2011-01-07T00:00:00.000Z
4    11490.0  11490.0  11129.0  11129.0  270360  2011-01-10T00:00:00.000Z
..       ...      ...      ...      ...     ...                       ...
243   6323.0   6721.0   6323.0   6468.0    7670  2011-12-26T00:00:00.000Z
244   6359.0   6359.0   6251.0   6251.0   59540  2011-12-27T00:00:00.000Z
245   6540.0   6540.0   6287.0   6432.0   32120  2011-12-28T00:00:00.000Z
246   6432.0   6468.0   6396.0   6468.0   11720  2011-12-29T00:00:00.000Z
247   6721.0   6721.0   6468.0   6468.0   19420  2011-12-30T00:00:00.000Z

[248 rows x 6 columns]


In [ ]:

jsonStr = json.dumps([{'open': 30689.0,
'high': 30689.0,
'low': 30689.0,
'close': 30700.0,
'volume': 3000,
'tradingDate': '2017-08-04T00:00:00.000Z'}])
print(jsonStr)

[{"open": 30689.0, "high": 30689.0, "low": 30689.0, "close": 30700.0, "volume": 3000, "tradingDate": "2017-08-04T00:00:00.000Z"}]


In [9]:
fd = int(time.mktime(time.strptime("2000-01-01", "%Y-%m-%d")))
td = int(time.mktime(time.strptime("2024-02-01", "%Y-%m-%d")))
size=0
for i,stock_symbol in enumerate(tickers[]):
  data = requests.get('https://apipubaws.tcbs.com.vn/stock-insight/v1/stock/bars-long-term?ticker={}&type=stock&resolution=D&from={}&to={}'.format(stock_symbol, fd, td)).text
  data = data[data.find("["):-1].replace("\"open\"", "\"ticker\":"+ "\""+ stock_symbol +"\""+","+"\"open\"")
  json.loads(data)
  with open("/content/stock/"+stock_symbol +".json", "w+") as outfile:
    json.dump(json.loads(data), outfile, ensure_ascii=False, indent = 4)
  size += os.path.getsize("/content/stock/"+stock_symbol +".json")
  # if size > 512*1048576:
  #   break
  print(str(1+1146+i) +"  " + stock_symbol +  "  " + str(size/1048576) + "  Expected:" + str(1630/1048576*size/(1146+i+1)))

1  VVS  0.016959190368652344  Expected:27.64348030090332
2  XDC  0.02595043182373047  Expected:21.149601936340332
3  HSV  0.11600685119628906  Expected:63.030389149983726
4  CST  0.2071218490600586  Expected:84.40215349197388
5  BVL  0.2908344268798828  Expected:94.8120231628418
6  SGI  0.36185455322265625  Expected:98.30382029215495
7  TOS  0.43325138092041016  Expected:100.88567870003837
8  VTZ  0.49300098419189453  Expected:100.44895052909851
9  SSH  0.5704412460327148  Expected:103.31324789259169
10  BCA  0.6420269012451172  Expected:104.6503849029541
11  FUCTVGF3  0.7073802947998047  Expected:104.82089822942561
12  FUEIP100  0.7754392623901367  Expected:105.33049980799358
13  GMH  0.8294315338134766  Expected:103.99795385507437
14  BIG  0.8837471008300781  Expected:102.89341245378766
15  CMM  0.8960647583007812  Expected:97.37237040201823
16  CNA  0.953669548034668  Expected:97.1550852060318
17  FUEKIV30  1.0086593627929688  Expected:96.7126330207376
18  ODE  1.0631628036499023  E

In [45]:
files=os.listdir("/content/stock")

def merge_JsonFiles(filename, output):
    result = list()
    for f1 in filename:
        with open("/content/stock/"+f1, 'r') as infile:
            result.extend(json.load(infile))

    with open('/content/'+output, 'w') as output_file:
        json.dump(result, output_file)

merge_JsonFiles(files,"final_stock_concat.json")

In [47]:
file_stats = os.stat("/content/final_stock_concat.json")

print(file_stats)
print(f'File Size in Bytes is {file_stats.st_size}')
print(f'File Size in MegaBytes is {file_stats.st_size / (1024 * 1024)}')

os.stat_result(st_mode=33188, st_ino=4588783, st_dev=47, st_nlink=1, st_uid=0, st_gid=0, st_size=529177187, st_atime=1675991307, st_mtime=1675995488, st_ctime=1675995488)
File Size in Bytes is 529177187
File Size in MegaBytes is 504.6626920700073


In [46]:
merge_JsonFiles(files[:20],"test.json")